### Wind Generation Forecasts 

https://transparency.entsoe.eu/generation/r2/dayAheadGenerationForecastWindAndSolar/show?name=&defaultValue=true&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=31.08.2021+00:00|CET|DAYTIMERANGE&dateTime.endDateTime=31.08.2021+00:00|CET|DAYTIMERANGE&area.values=CTY|10YIE-1001A00010!BZN|10Y1001A1001A59C&productionType.values=B16&productionType.values=B18&productionType.values=B19&processType.values=A18&processType.values=A01&processType.values=A40&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)

### Actual Wind Generation

https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show

In [1]:
# import libraries
import numpy as np
import pandas as pd
from scipy.stats import iqr
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import plot_importance, plot_tree
import shap
import datetime as dt
import time
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import scipy.stats
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
import xgboost
import random

import yellowbrick
from yellowbrick.regressor import ResidualsPlot, PredictionError
from yellowbrick.regressor import residuals_plot


import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
#plotting size
#plt.rcParams['figure.figsize'] = [10, 4]
#plt.rcParams['figure.dpi'] = 100
#plt.style.use('ggplot')


sns.set_style('whitegrid')
plt.rcParams['figure.figsize']=(20,10) # for graphs styling
plt.style.use('tableau-colorblind10') # for graph stying

In [2]:
# data locations
files_loc = "Wind_Data/"
# files start with
files_actual = "Actual Generation per Production Type_"
files_forecast = "Generation Forecasts for Wind and Solar_"

## Read Files

### Read actual data

In [3]:
# read actual data files for all years and return single dataframe
# containing date and data(MW) for all the years
def prepare_actual(files_loc, files_actual):
  # create empty dataframe to be appended wih data from each csv
  actual_whole = pd.DataFrame(columns = ["Date", "MW"])
  for year in [2016, 2017, 2018, 2019, 2020]:
    # get file name and read the file
    input_file = files_loc + files_actual + str(year) + ".csv"
    df = pd.read_csv(input_file, header = 0)
    # remove duplicates
    df = df.drop_duplicates()
    # rename column
    df = df.rename(columns={"Wind Onshore  - Actual Aggregated [MW]": "MW"})
    # get date from MTU column
    df['Date'] = pd.to_datetime(df['MTU'].str[:13], dayfirst=True)
    # we don't need times (only date is enough), because we are going to aggregate data by day anyway
    # append the data to the whole data variable
    actual_whole = actual_whole.append(df[["Date","MW"]])
  return (actual_whole)
# call the function
df_actual = prepare_actual(files_loc, files_actual)

In [4]:
# read actual data files for all years and return single dataframe
# containing date and data(MW) for all the years
def prepare_actual(files_loc, files_actual):
  # create empty dataframe to be appended wih data from each csv
  actual_whole = pd.DataFrame(columns = ["Date", "MW"])
  for year in [2021]:
    # get file name and read the file
    input_file = files_loc + files_actual + str(year) + ".csv"
    df = pd.read_csv(input_file, header = 0)
    # remove duplicates
    df = df.drop_duplicates()
    # rename column
    df = df.rename(columns={"Wind Onshore  - Actual Aggregated [MW]": "MW"})
    # get date from MTU column
    df['Date'] = pd.to_datetime(df['MTU'].str[:13], dayfirst=True)
    # we don't need times (only date is enough), because we are going to aggregate data by day anyway
    # append the data to the whole data variable
    actual_whole = actual_whole.append(df[["Date","MW"]])
  return (actual_whole)
# call the function
df_external_actual = prepare_actual(files_loc, files_actual)

### Read forecast data

In [5]:
# read forecast data files for all years and return single dataframe
# containing date and data(MW) for all the years
def prepare_forecast(files_loc, files_forecast): 
  # create empty dataframe to be appended wih data from each csv
  forecast_whole = pd.DataFrame(columns = ["Date", "MW"])
  for year in [2016, 2017, 2018, 2019, 2020]:
    # get file name and read the file
    input_file = files_loc + files_forecast + str(year) + ".csv"
    df = pd.read_csv(input_file, header = 0)
    # remove duplicates
    df = df.drop_duplicates()
    # rename column
    df = df.rename(columns={"Generation - Wind Onshore  [MW] Day Ahead/ BZN|IE(SEM)": "MW"})
    # get date from MTU column
    df['Date'] = pd.to_datetime(df['MTU (CET)'].str[:13], dayfirst=True)
    # we don't need times (only date is enough), because we are going to aggregate data by day anyway
    # append the data to the whole data variable
    forecast_whole = forecast_whole.append(df[["Date","MW"]])
  return (forecast_whole)
df_forecast = prepare_forecast(files_loc, files_forecast)

In [6]:
# read forecast data files for all years and return single dataframe
# containing date and data(MW) for all the years
def prepare_forecast(files_loc, files_forecast): 
  # create empty dataframe to be appended wih data from each csv
  forecast_whole = pd.DataFrame(columns = ["Date", "MW"])
  for year in [2021]:
    # get file name and read the file
    input_file = files_loc + files_forecast + str(year) + ".csv"
    df = pd.read_csv(input_file, header = 0)
    # remove duplicates
    df = df.drop_duplicates()
    # rename column
    df = df.rename(columns={"Generation - Wind Onshore  [MW] Day Ahead/ Ireland (IE)": "MW"})
    # get date from MTU column
    df['Date'] = pd.to_datetime(df['MTU (CET)'].str[:13], dayfirst=True)
    # we don't need times (only date is enough), because we are going to aggregate data by day anyway
    # append the data to the whole data variable
    forecast_whole = forecast_whole.append(df[["Date","MW"]])
  return (forecast_whole)
df_external_forecast = prepare_forecast(files_loc, files_forecast)

## Custom Results

In [7]:
# make custom scorer
from sklearn.metrics import make_scorer

def rmse(actual, predict):
    predict = np.array(predict)
    actual = np.array(actual)
    distance = predict - actual
    square_distance = distance ** 2
    mean_square_distance = square_distance.mean()
    score = np.sqrt(mean_square_distance)
    
    return score

rmse_score = make_scorer(rmse, greater_is_better = True)

In [8]:
import sklearn.metrics as metrics
def regression_results(y_true, y_pred):
    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)
    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [9]:
# define a MAPE function
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

mape_score = make_scorer(mape, greater_is_better = True)

In [10]:
df_actual_nona = df_actual.fillna(method="ffill")

In [11]:
df_forecast_nona = df_forecast.fillna(method="ffill")

## Aggregate Data by Hour and Drop NAs

In [12]:
df_actual_hour = df_actual_nona.resample('60min', on='Date').mean()

In [13]:
df_actual_hour['Date'] = df_actual_hour.index
df_actual_hour['Date'] = pd.to_datetime(df_actual_hour['Date'])

In [14]:
df_forecast_hour = df_forecast_nona.resample('60min', on='Date').mean()

## Add Moving Averages as variables

In [15]:
# add simple moving average (aggregate MW) of lags 4 and 7
df_actual_hour['SMA_2'] = df_actual_hour.iloc[:,0].rolling(window=48,min_periods=1).mean()
df_actual_hour['SMA_7'] = df_actual_hour.iloc[:,0].rolling(window=168,min_periods=1).mean()

# add exponential moving average
df_actual_hour['EMA_7'] = df_actual_hour.iloc[:,0].ewm(halflife=28,adjust=False).mean()
df_actual_hour['EMA_28'] = df_actual_hour.iloc[:,0].ewm(halflife=672,adjust=False).mean()

In [16]:
# Let's log transform some variables
df_actual_hour['SMA_2'] = df_actual_hour['SMA_2'].transform([np.log])
df_actual_hour['SMA_7'] = df_actual_hour['SMA_7'].transform([np.log])
df_actual_hour['EMA_7'] = df_actual_hour['EMA_7'].transform([np.log])
df_actual_hour['EMA_28'] = df_actual_hour['EMA_28'].transform([np.log])
#df_actual_agg['emedian'] = df_actual_agg['median'].transform([np.log])

## Extra adjustment - add trend variables

In [17]:
# add a trend variable equal to the "row number" as the data is ordered in time
df_actual_hour["trend"] = np.arange(df_actual_hour.shape[0])

In [18]:
# add month as a dummy variable since we clearly have seasonality.
month_dummy = pd.get_dummies(df_actual_hour.index.month)
# drop one month to for linear regression to work
month_dummy = month_dummy.drop(columns=[12])

In [19]:
# # add to df_actual_agg
df_actual_hour = df_actual_hour.reset_index(drop=True)
df_actual_hour = pd.concat([df_actual_hour, month_dummy], axis=1)

## Format Data as X Y for usage in models

In [20]:
df_actual_hour.rename(columns={"MW": "Hourly_MW", 
                               "Date" : "Date",
                              "SMA_2": "actual_log_SMA_2", 
                              "SMA_7": "actual_log_SMA_7", 
                              "EMA_7": "EMA_7", 
                              "EMA_28": "actual_log_EMA_28",
                               "trend" : "Seasonal Trend",
                              1: "month_1", 
                              2: "month_2", 
                              3: "month_3", 
                              4: "month_4", 
                              5: "month_5", 
                              6: "month_6", 
                              7: "month_7", 
                              8: "month_8", 
                              9: "month_9", 
                              10: "month_10", 
                              11: "month_11",
                              }, errors="raise")


,Hourly_MW,Date,actual_log_SMA_2,actual_log_SMA_7,EMA_7,actual_log_EMA_28,Seasonal Trend,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11
0,1061.5,2016-01-01 00:00:00,6.967438,6.967438,6.967438,6.967438,0,1,0,0,0,0,0,0,0,0,0,0
1,1004.0,2016-01-01 01:00:00,6.939980,6.939980,6.966113,6.967382,1,1,0,0,0,0,0,0,0,0,0,0
2,957.5,2016-01-01 02:00:00,6.915393,6.915393,6.963744,6.967281,2,1,0,0,0,0,0,0,0,0,0,0
3,1069.0,2016-01-01 03:00:00,6.930495,6.930495,6.964008,6.967289,3,1,0,0,0,0,0,0,0,0,0,0
4,1063.0,2016-01-01 04:00:00,6.938284,6.938284,6.964126,6.967291,4,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43843,3196.5,2020-12-31 19:00:00,7.033764,7.668932,7.534834,7.525780,43843,0,0,0,0,0,0,0,0,0,0,0
43844,3060.5,2020-12-31 20:00:00,7.061522,7.675184,7.550235,7.526450,43844,0,0,0,0,0,0,0,0,0,0,0
43845,2976.0,2020-12-31 21:00:00,7.092600,7.681912,7.563964,7.527071,43845,0,0,0,0,0,0,0,0,0,0,0
43846,2892.0,2020-12-31 22:00:00,7.125652,7.689061,7.576125,7.527645,43846,0,0,0,0,0,0,0,0,0,0,0


In [21]:
# # don't use first values as their moving averages are not correct
y = df_actual_hour.iloc[168:-1,0]
y = y.to_frame()
y = y.reset_index(drop=True)
y

,MW
0,787.0
1,629.5
2,508.0
3,365.0
4,262.5
...,...
43674,3323.0
43675,3196.5
43676,3060.5
43677,2976.0


### Create X variable

In [23]:
# Use values starting at 5 of actual data and values starting at 7 of forecast data.
X = df_actual_hour.iloc[120:-49]

# add columns from "forecast" data
# make sure indexes match when adding data
X = X.reset_index(drop=True)
X["pred_sum"] = df_forecast_hour.iloc[168:-1,0].to_frame().reset_index(drop=True)
X = X.drop(['Date'], axis=1)
X

,MW,SMA_2,SMA_7,EMA_7,EMA_28,trend,1,2,3,4,5,6,7,8,9,10,11,pred_sum
0,444.0,6.494132,6.897722,6.725736,6.958391,120,1,0,0,0,0,0,0,0,0,0,0,988.0
1,375.5,6.489632,6.892621,6.712208,6.957728,121,1,0,0,0,0,0,0,0,0,0,0,1069.0
2,259.0,6.480508,6.886611,6.695315,6.956951,122,1,0,0,0,0,0,0,0,0,0,0,979.0
3,204.0,6.469250,6.880206,6.676864,6.956120,123,1,0,0,0,0,0,0,0,0,0,0,885.0
4,111.5,6.454674,6.873098,6.655623,6.955198,124,1,0,0,0,0,0,0,0,0,0,0,792.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43674,2331.0,8.066093,7.834306,7.975537,7.544943,43794,0,0,0,0,0,0,0,0,0,0,0,1746.0
43675,2016.0,8.061263,7.835985,7.968003,7.545011,43795,0,0,0,0,0,0,0,0,0,0,0,1696.0
43676,1528.0,8.049314,7.836097,7.956426,7.544813,43796,0,0,0,0,0,0,0,0,0,0,0,1639.0
43677,1209.0,8.034570,7.835387,7.942233,7.544441,43797,0,0,0,0,0,0,0,0,0,0,0,1592.0


### Train/Test Split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
start = time.time()


model = KNeighborsRegressor()


param_search = { 
    'n_neighbors':[2,3,4,5,6,7,8,9,10],
    'weights':['uniform', 'distance'],
    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [i for i in range(1,10)],
    'n_jobs':[-1]
    #'p' : [1,2]
}

#tscv = TimeSeriesSplit(n_splits=5)
kfold = model_selection.KFold(n_splits=3, random_state=seed)
gsearch = GridSearchCV(estimator=model, cv=kfold, param_grid=param_search, scoring = mape_score)
gsearch.fit(X_train, y_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

end = time.time()

knn_tuning = end-start
print ("Time elapsed:", knn_tuning)

In [ ]:
gsearch.best_estimator_

In [ ]:
y_true = y_test.values
y_pred = best_model.predict(X_test)
print('KNN Regressor')
print('MAPE for KNN Regressor on internal validation data :', round(mape(y_true,y_pred),2))
print('RMSE for KNN Regressor on internal validation data :', round(rmse(y_true,y_pred),2))

In [ ]:
knn_predictions = best_model.predict(X)
plt.plot(y[:100], marker='o')
plt.plot(knn_predictions[:100], marker='o');
plt.plot(X['pred_sum'][:100], marker='o');
plt.legend(["Actual","Model Prediction","Original Prediction"])
plt.title("KNN Regressor Predictions: First 100 Hours");
plt.ylabel("MW")
plt.xlabel("Hour");

In [ ]:
model = KNeighborsRegressor(algorithm = 'auto', leaf_size = 1, n_jobs = -1, n_neighbors = 10, weights = 'uniform')
visualizer = ResidualsPlot(model)
visualizer.fit(X_train, y_train) 
visualizer.score(X_test, y_test) 
g = visualizer.poof()